In [ ]:
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().setAppName('project')
sc = SparkContext(conf=conf)
sc

In [22]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('project').config('spark.some.config.option', "some-value").getOrCreate()

In [23]:
df_float = spark.read.csv('human_activity_raw_sensor_data/sensor_sample_float.csv', header=True)
df_int = spark.read.csv('human_activity_raw_sensor_data/sensor_sample_int.csv', header=True)
df3 = spark.read.csv('human_activity_raw_sensor_data/sensor.csv', header=True)

In [24]:
import datetime
from pyspark.sql.functions import *
df_int = df_int.withColumn("timestamp",expr("substring(timestamp, 1, length(timestamp)-7)"))
df_float = df_float.withColumn("timestamp",expr("substring(timestamp, 1, length(timestamp)-7)"))

In [25]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [26]:
df_int = df_int\
    .withColumn('year', year(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('month', month(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('date', dayofmonth(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('hour', hour(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('minute', minute(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('second', second(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('week', weekofyear(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('weekday', dayofweek(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))


In [27]:
df_float = df_float\
    .withColumn('year', year(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('month', month(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('date', dayofmonth(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('hour', hour(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('minute', minute(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('second', second(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('week', weekofyear(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))\
    .withColumn('weekday', dayofweek(to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss')))


In [28]:
df_int = df_int.join(df3,['sensor_id'],how='inner').drop(*["node_id", "type"])
df_float = df_float.join(df3,['sensor_id'],how='inner').drop(*["node_id", "type"])

In [29]:
df_int = df_int.withColumn('room', split(df_int['name'], '/').getItem(0)) \
        .withColumn('device', split(df_int['name'], '/').getItem(1)) \
        .withColumn('sensor_type', split(df_int['name'], '/').getItem(2))

df_float = df_float.withColumn('room', split(df_float['name'], '/').getItem(0)) \
        .withColumn('device', split(df_float['name'], '/').getItem(1)) \
        .withColumn('sensor_type', split(df_float['name'], '/').getItem(2))

In [30]:
df_int = df_int.drop("name")
df_float = df_float.drop("name")

In [31]:
from pyspark.sql.types import *
df_int = df_int.withColumn('value', col('value').cast(IntegerType()))
df_float = df_float.withColumn('value', col('value').cast(FloatType()))

In [32]:
df_int.show()

+---------+--------+-------------------+-----+----+-----+----+----+------+------+----+-------+----------+--------+-----------+
|sensor_id|value_id|          timestamp|value|year|month|date|hour|minute|second|week|weekday|      room|  device|sensor_type|
+---------+--------+-------------------+-----+----+-----+----+----+------+------+----+-------+----------+--------+-----------+
|     5892|18730541|2020-02-26 00:00:00|    0|2020|    2|  26|   0|     0|     0|   9|      4|   bedroom|ambience|     motion|
|     5887|18730542|2020-02-26 00:00:00| 1024|2020|    2|  26|   0|     0|     0|   9|      4|   kitchen|   stove|      light|
|     5891|18730544|2020-02-26 00:00:00|    0|2020|    2|  26|   0|     0|     0|   9|      4|livingroom|ambience|     motion|
|     5896|18730546|2020-02-26 00:00:00|  652|2020|    2|  26|   0|     0|     0|   9|      4|   bedroom|     bed|   pressure|
|     6127|18730548|2020-02-26 00:00:00| 1024|2020|    2|  26|   0|     0|     0|   9|      4|livingroom|      

In [33]:
df_float.show()

+---------+--------+-------------------+-----+----+-----+----+----+------+------+----+-------+--------+--------+-----------+
|sensor_id|value_id|          timestamp|value|year|month|date|hour|minute|second|week|weekday|    room|  device|sensor_type|
+---------+--------+-------------------+-----+----+-----+----+----+------+------+----+-------+--------+--------+-----------+
|     6222|  315318|2020-02-26 11:22:36|17.78|2020|    2|  26|  11|    22|    36|   9|      4|bathroom|ambience|   humidity|
|     6223|  315319|2020-02-26 11:22:36|26.51|2020|    2|  26|  11|    22|    36|   9|      4|bathroom|ambience|temperature|
|     6223|  315320|2020-02-26 19:36:36|27.27|2020|    2|  26|  19|    36|    36|   9|      4|bathroom|ambience|temperature|
|     6222|  315321|2020-02-26 19:36:36|51.52|2020|    2|  26|  19|    36|    36|   9|      4|bathroom|ambience|   humidity|
|     6223|  315322|2020-02-26 19:36:37|27.26|2020|    2|  26|  19|    36|    37|   9|      4|bathroom|ambience|temperature|


In [34]:
df3.show()

+---------+-------+-----+--------------------+
|sensor_id|node_id| type|                name|
+---------+-------+-----+--------------------+
|     5894| 541150|  INT|corridor/ambience...|
|     5895| 541151|  INT|bathroom/ambience...|
|     7125| 542381|  INT|bathroom/ambience...|
|     5896| 541152|  INT|bedroom/bed/pressure|
|     6127| 541383|  INT| livingroom/tv/light|
|     6220| 541444|  INT| balcon/door/contact|
|     6253| 541509|  INT|kitchen/fridge/co...|
|     6632| 541888|FLOAT|kitchen/coffeemak...|
|     6633| 541889|FLOAT|kitchen/sandwichm...|
|     6634| 541890|FLOAT|kitchen/dishwashe...|
|     6635| 541891|FLOAT|kitchen/kettle/cu...|
|     6636| 541892|FLOAT|bathroom/washingm...|
|     6896| 542152|FLOAT|kitchen/microwave...|
|     6686| 541885|  INT|bedroom/ambience_...|
|     6687| 541883|  INT|bedroom/weightsca...|
|     7139| 542395|FLOAT|corridor/ilifeRob...|
|     6222| 541141|FLOAT|bathroom/ambience...|
|     6223| 541141|FLOAT|bathroom/ambience...|
|     5887| 5

In [35]:
df3_int =df3[df3.type == "INT"]
df3_float = df3[df3.type == "FLOAT"]

In [36]:
df3_float.show(truncate=False)

+---------+-------+-----+-------------------------------+
|sensor_id|node_id|type |name                           |
+---------+-------+-----+-------------------------------+
|6632     |541888 |FLOAT|kitchen/coffeemaker/current    |
|6633     |541889 |FLOAT|kitchen/sandwichmaker/current  |
|6634     |541890 |FLOAT|kitchen/dishwasher/current     |
|6635     |541891 |FLOAT|kitchen/kettle/current         |
|6636     |541892 |FLOAT|bathroom/washingmachine/current|
|6896     |542152 |FLOAT|kitchen/microwave/current      |
|7139     |542395 |FLOAT|corridor/ilifeRobot/current    |
|6222     |541141 |FLOAT|bathroom/ambience/humidity     |
|6223     |541141 |FLOAT|bathroom/ambience/temperature  |
+---------+-------+-----+-------------------------------+



In [41]:
df3_int.show(truncate=False)

+---------+-------+----+-------------------------------------+----------+----------------------+-----------+
|sensor_id|node_id|type|name                                 |room      |device                |sensor_type|
+---------+-------+----+-------------------------------------+----------+----------------------+-----------+
|5894     |541150 |INT |corridor/ambience/motion             |corridor  |ambience              |motion     |
|5895     |541151 |INT |bathroom/ambience/motion             |bathroom  |ambience              |motion     |
|7125     |542381 |INT |bathroom/ambience/light              |bathroom  |ambience              |light      |
|5896     |541152 |INT |bedroom/bed/pressure                 |bedroom   |bed                   |pressure   |
|6127     |541383 |INT |livingroom/tv/light                  |livingroom|tv                    |light      |
|6220     |541444 |INT |balcon/door/contact                  |balcon    |door                  |contact    |
|6253     |541509 |

In [37]:
df3_int = df3_int.withColumn('room', split(df3_int['name'], '/').getItem(0)) \
        .withColumn('device', split(df3_int['name'], '/').getItem(1)) \
        .withColumn('sensor_type', split(df3_int['name'], '/').getItem(2))
df3_float = df3_float.withColumn('room', split(df3_float['name'], '/').getItem(0)) \
        .withColumn('device', split(df3_float['name'], '/').getItem(1)) \
        .withColumn('sensor_type', split(df3_float['name'], '/').getItem(2))

In [18]:
for row in df3_float.rdd.collect():
    df = df_float[df_float.room == row.room]
    df = df[df.device == row.device]
    df = df[df.sensor_type == row.sensor_type]
#df_float2.rdd.map(lambda x: x.write.save(path='float_sensor/kitchen-coffemaker.csv', format='csv', mode='append', sep='\t'))
    df.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('float_sensor/'+row.room+'-'+row.device+'-'+row.sensor_type+'.csv')

AnalysisException: path file:/C:/Users/lerjebo/Desktop/DataMining-Project-main/float_sensor/bathroom-ambience.csv already exists.;

In [39]:
for row in df3_int.rdd.collect():
    df = df_int[df_int.room == row.room]
    df = df[df.device == row.device]
    df = df[df.sensor_type == row.sensor_type]
#df_float2.rdd.map(lambda x: x.write.save(path='float_sensor/kitchen-coffemaker.csv', format='csv', mode='append', sep='\t'))
    df.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('int_sensor/'+row.room+'-'+row.device+'-'+row.sensor_type+'.csv')

In [43]:
df1 = spark.read.csv('int_sensor/livingroom-tv-light.csv', header=True)

In [44]:
df1.count()

14865503

In [45]:
df_int1 = df_int[df_int.room == "livingroom"]
df_int1 = df_int1[df_int1.device == "tv"]
df_int1 = df_int1[df_int1.sensor_type == "light"]

In [46]:
df_int1.count()

14865503

In [47]:
df_int.count()

186824861